In [9]:
import numpy as np 
import pandas as pd 
import tensorflow as tf
import os
import opendatasets as od
from keras.preprocessing.image import ImageDataGenerator as IDG, img_to_array, load_img

import warnings
warnings.filterwarnings("ignore")


In [10]:
od.download('https://www.kaggle.com/datasets/sanikamal/horses-or-humans-dataset')
path = 'horses-or-humans-dataset'

Skipping, found downloaded files in ".\horses-or-humans-dataset" (use force=True to force download)


In [11]:
def Conv2D_and_MaxPooling2D(model, num_convolution, size_kernel):
    model.add(tf.keras.layers.Conv2D(num_convolution, size_kernel, activation='relu'))
    model.add(tf.keras.layers.MaxPooling2D(2, 2))

In [12]:
'''
• Rotating each image randomly up to 40 degrees left or right
• Translating the image up to 20% vertically or horizontally
• Shearing the image by up to 20%
• Zooming the image by up to 20%
• Randomly flipping the image horizontally or vertically
• Filling in any missing pixels after a move or shear with nearest neighbors
'''

train_datagen = IDG(
    rescale = 1/255,
    rotation_range = 40,
    width_shift_range = 0.2,
    height_shift_range = 0.2,
    shear_range = 0.2,
    zoom_range = 0.2,
    horizontal_flip = True,
    fill_mode = 'nearest'
)

In [13]:
train_generator = train_datagen.flow_from_directory(path + '/train', 
                                                    target_size=(300, 300), 
                                                    class_mode='binary')

model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Conv2D(16, (3, 3), activation='relu', input_shape=(300, 300, 3)))
model.add(tf.keras.layers.MaxPooling2D(2,2))
Conv2D_and_MaxPooling2D(model, 32, (3, 3))
for i in range(3):
    Conv2D_and_MaxPooling2D(model, 64, (3, 3))
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(512, activation='relu'))
model.add(tf.keras.layers.Dense(1, activation='sigmoid'))
print('Add layers completely')

model.compile(loss='binary_crossentropy',
             optimizer=tf.keras.optimizers.RMSprop(lr=0.001),
             metrics=['accuracy'])

validation_datagen = IDG(rescale=1/255)

validation_generator = validation_datagen.flow_from_directory(
    path + '/validation',
    target_size=(300, 300),
    class_mode = 'binary'
)

history = model.fit_generator(train_generator, 
                             epochs=15,
                             validation_data=validation_generator)

Found 1027 images belonging to 2 classes.


Add layers completely
Found 256 images belonging to 2 classes.


C:\Users\qhuy9\AppData\Local\Temp\ipykernel_11612\1190071149.py:28: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  history = model.fit_generator(train_generator,


Epoch 1/15



33/33 [==============================] - 54s 2s/step - loss: 0.6896 - accuracy: 0.5268 - val_loss: 0.6883 - val_accuracy: 0.5547
Epoch 2/15
33/33 [==============================] - 38s 1s/step - loss: 0.7248 - accuracy: 0.6310 - val_loss: 0.5904 - val_accuracy: 0.5781
Epoch 3/15
33/33 [==============================] - 34s 1s/step - loss: 0.5548 - accuracy: 0.7215 - val_loss: 1.9524 - val_accuracy: 0.5000
Epoch 4/15
33/33 [==============================] - 33s 996ms/step - loss: 0.5858 - accuracy: 0.7303 - val_loss: 0.6882 - val_accuracy: 0.7695
Epoch 5/15
33/33 [==============================] - 35s 1s/step - loss: 0.5115 - accuracy: 0.7848 - val_loss: 1.5160 - val_accuracy: 0.6133
Epoch 6/15
33/33 [==============================] - 38s 1s/step - loss: 0.3840 - accuracy: 0.8423 - val_loss: 1.8003 - val_accuracy: 0.6328
Epoch 7/15
33/33 [==============================] - 33s 983ms/step - loss: 0.3401 - accuracy: 0.8656 - val_loss: 1.0530 - val_accuracy: 0.8242
Epoch 8/15
33/33 [=======

In [14]:
path_image = 'images'
files = os.listdir(path_image)

for file in files:
    img = load_img(path_image + '/' + file, 
                         target_size=(300, 300))
    x = img_to_array(img)
    x = np.expand_dims(x, axis=0)
    
    image_tensor = np.vstack([x])
    classes = model.predict(image_tensor)
    print(classes)
    print(classes[0])
    
    if classes[0] > 0.5:
        print(file + " is a human")
    else:
        print(file + " is a horse")

1/1 [==============================] - 0s 410ms/step
[[1.]]
[1.]
horse-197199_640.jpg is a human
1/1 [==============================] - 0s 39ms/step
[[1.]]
[1.]
woman-1274056_640.jpg is a human
1/1 [==============================] - 0s 38ms/step
[[1.]]
[1.]
woman-1439909_640.jpg is a human
1/1 [==============================] - 0s 37ms/step
[[1.]]
[1.]
woman-1754895_640.jpg is a human
1/1 [==============================] - 0s 32ms/step
[[1.]]
[1.]
woman-1867093_640.jpg is a human


: 